# TODO:
1. animate grpahs using program time

In [2]:
# add package to path to import app
# import sys 
# sys.path.append("./atscale_dash")
from atscale_dash import app

In [ ]:
app.run_server(jupyter_mode='tab', debug=True)

export filtered datafarme
